In [1]:
import json 
import re

with open("./chatbot_train.json", 'r') as f:
    data = json.load(f)

for i, d in enumerate(data):
    if d['title'].strip() == "":
        data[i]['title'] = "제목"
    if "$" in d['title']:
        data[i]['title'] = d['title'].replace("$", "")
    if "달러" in d['title']:
        data[i]['title'] = d['title'].replace("달러", "원")
    for j, e in enumerate(d['events']):
        if e['message'].strip() == "":
            data[i]['events'][j]['message'] = "네"
        if "OFFER" in e['message'] and e['role'] == "구매자":
            price = re.findall(r"\d+", e['message'])[0]
            data[i]['events'][j]['message'] = f"{price}원에 구매할게요."
        elif "OFFER" in e['message'] and e['role'] == "판매자":
            price = re.findall(r"\d+", e['message'])[0]
            data[i]['events'][j]['message'] = f"그럼 {price}원은 어떠세요?"
        if "달러" in e['message']:
            data[i]['events'][j]['message'] = e['message'].replace("달러", "원")
        if "$" in e['message']:
            data[i]['events'][j]['message'] = e['message'].replace("$", "")
        if "ACCEPT" in e['message']:
            data[i]['events'][j]['message'] = '좋습니다. 수락할게요.'
        if "REJECT" in e['message']:
            data[i]['events'][j]['message'] = '거절하겠습니다.'

with open("./new.json", "w", encoding='utf-8') as f:
    f.write(
        json.dumps(data, indent=4, ensure_ascii=False)
    )
print('done')

done


In [2]:
print(len(data))

3062


In [1]:
# to hub
from datasets import Dataset
import json 
from huggingface_hub import notebook_login
notebook_login(new_session=False)
 
with open("translated_new.json","r") as f:
    data = json.load(f)

dataset = Dataset.from_json('translated_new.json')
dataset.push_to_hub("ggul-tiger/negobot-translated-train-3062")



Found cached dataset json (/opt/ml/.cache/huggingface/datasets/json/default-9f62e95e489b36ca/0.0.0)


User is already logged in.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.


In [37]:
import json 
with open("gpt4_53.json", "r", encoding='utf-8') as f:
    data = json.load(f)

for i, d in enumerate(data):
    # if 'Title' in d.keys():
    #     d['title'] = d['Title']
    #     del d['Title']
    # if 'Price' in d.keys():
    #     d['price'] = d['Price']
    #     del d['Price']
    # if '만원' in str(d['price']):
    #     d['price'] = d['price'].replace('만원', '0000').strip()
    #     d['price'] = int(d['price'].replace(" ", ""))
    for j, e in enumerate(d['events']):
        if "Buyer" in e['role']:
            data[i]['events'][j]['role'] = '구매자'
        else:
            data[i]['events'][j]['role'] = '판매자'
        if j == 0 and e['role'] == '판매자':
            print(i)
            data[i]['events'].insert(0, {'role':'구매자', 'message':'안녕하세요.'})

53
54


In [38]:
data[0]['events']

[{'role': '구매자', 'message': '안녕하세요! LG 그램 13인치 노트북에 관심이 있어요. 가격 조정 가능할까요?'},
 {'role': '판매자', 'message': '안녕하세요! 네, 가격 조정은 얼마 정도를 생각하시나요?'},
 {'role': '구매자', 'message': '혹시 60만 원까지는 괜찮을까요? 사용기간과 키보드 상태를 고려해서 요청하는 건데요.'},
 {'role': '판매자',
  'message': '제안해주신 가격은 너무 낮아서 어렵습니다. 하지만, 제안한 금액에서 조금만 올리면 수락할 수 있을 것 같네요.'},
 {'role': '구매자', 'message': '그럼 65만 원으로 할게요. 문제점 때문에 네고를 한 건 아닙니다.'},
 {'role': '판매자', 'message': '좋아요, 그 가격이면 거래 가능합니다. 65만 원으로 합의하고 진행할까요?'},
 {'role': '구매자', 'message': '네, 그렇게 하죠. 어디서 택배로 보내주시나요?'},
 {'role': '판매자',
  'message': '저희는 서울에 있어서 서울 지역이면 직거래도 가능합니다. 아니면 택배로 보낼 수도 있습니다.'},
 {'role': '구매자', 'message': '저는 다른 지역에 사는데 택배로 보내주세요.'},
 {'role': '판매자', 'message': '알겠습니다. 제가 안전하게 포장해서 발송해드릴게요.'}]

In [39]:
with open("gpt4_kor.json", "w", encoding='utf-8') as f:
    f.write(
        json.dumps(data, indent=4, ensure_ascii=False)
    )

In [40]:
from datasets import Dataset 

dataset = Dataset.from_json('gpt4_kor.json')
dataset.push_to_hub("dnjdsxor21/nego-dialogue-53", split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /opt/ml/.cache/huggingface/datasets/json/default-c3c9e6824ce0fe32/0.0.0. Subsequent calls will reuse this data.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.
